In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd

plt.style.use('fivethirtyeight')

from sklearn.metrics import mean_squared_error
from keras.layers import Dense, Activation, Dropout  
from keras.layers import LSTM
from keras.models import Sequential
from datetime import datetime
from binance.client import Client 
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")

client = Client(api_key, api_secret)

In [ ]:
candles = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_5MINUTE, limit=1000)

In [ ]:
len(candles)

In [ ]:
candles[999]

In [ ]:
price = np.array([float(candles[i][4]) for i in range(1000)])

In [ ]:
time = np.array([int(candles[i][0]) for i in range(1000)])

t = np.array([datetime.fromtimestamp(time[i]/1000).strftime('%H:%M:%S') for i in range(1000)])

In [ ]:
price.shape
print(price)

In [ ]:
plt.figure(figsize=(8,5))
plt.xlabel('Time Step')
plt.ylabel('Bitcoin Price $')
plt.plot(price)

In [ ]:
timeframe = pd.DataFrame({'Time':t,'Price $BTC':price})
timeframe

In [ ]:
price = price.reshape(1000,1)
print(price)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


In [ ]:
scaler.fit(price[:750])

In [ ]:
price = scaler.transform(price)

In [ ]:
df = pd.DataFrame(price.reshape(200,5),columns=['First','Second','Third','Fourth','Target'])

In [ ]:
df.head()

## Train Test Split


In [ ]:
x_train = df.iloc[:149,:4]
y_train = df.iloc[:149,-1]

x_test = df.iloc[145:199,:4]
y_test = df.iloc[145:199,-1]

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test  = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
x_train.shape , x_test.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input

model = Sequential()

model.add(Input(shape=(4, 1)))

model.add(LSTM(40, return_sequences=True))

model.add(LSTM(80, return_sequences=False))

model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='rmsprop')

model.summary()

In [ ]:
model.fit(x_train, y_train, batch_size=5,epochs=500)

In [ ]:
y_pred = model.predict(x_test)
print(y_pred)

In [ ]:
plt.figure(figsize=[8,5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Normalized Price')
plt.plot(y_test, label='True')
plt.plot(y_pred, label='Prediction')
plt.legend()

In [ ]:
y_test_reshaped = y_test.reshape(-1, 1)
y_pred_reshaped = y_pred.reshape(-1, 1)

y_test_inverse = scaler.inverse_transform(y_test_reshaped)
y_pred_inverse = scaler.inverse_transform(y_pred_reshaped)

y_test_inverse = y_test_inverse.flatten()
y_pred_inverse = y_pred_inverse.flatten()

plt.figure(figsize=[8, 5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.plot(y_test_inverse, label='True')
plt.plot(y_pred_inverse, label='Prediction')
plt.legend()
plt.show()

In [ ]:
testScore = np.sqrt(mean_squared_error(y_test_reshaped, y_pred_reshaped))
print('Test Score: %.2f RMSE' % testScore)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
print('RSquared :','{:.2%}'.format(r2_score(y_test,y_pred)))

In [ ]:
model.save("Bitcoin_model.h5")

## Second Model

In [ ]:
from sklearn.svm import SVR

In [ ]:
trainX = df.iloc[:149,:4]
trainY = df.iloc[:149,-1]

testX = df.iloc[150:199,:4]
testY = df.iloc[150:199,-1]

In [ ]:
svr_linear = SVR(kernel='linear',C=1e3, gamma=0.1)
svr_linear.fit(trainX,trainY)

In [ ]:
predY = svr_linear.predict(testX) 
print(predY)

In [ ]:
testY_array = np.array(testY).reshape(-1, 1)
predY_array = np.array(predY).reshape(-1, 1)

testY_inverse = scaler.inverse_transform(testY_array)
predY_inverse = scaler.inverse_transform(predY_array)

testY_inverse = testY_inverse.flatten()
predY_inverse = predY_inverse.flatten()

plt.figure(figsize=[8, 5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.plot(testY_inverse, label='True')
plt.plot(predY_inverse, label='Prediction')
plt.legend()
plt.show()

In [ ]:
testY_array = np.array(testY).reshape(-1, 1)
predY_array = np.array(predY).reshape(-1, 1)

testY_inverse = scaler.inverse_transform(testY_array)
predY_inverse = scaler.inverse_transform(predY_array)

testScore = np.sqrt(mean_squared_error(testY_inverse, predY_inverse))
print('Test Score: %.2f RMSE' % (testScore))


In [ ]:
print('RSquared :','{:.2%}'.format(r2_score(testY,predY)))

## Hyperparameter tuning


In [ ]:
param_grid = {"C": [1e-2,1e-1,1e0, 1e1, 1e2, 1e3, 1e4],
              "gamma": np.logspace(-2, 2, 50),
             'epsilon':[0.1,0.2,0.5,0.3]}

from sklearn.model_selection import RandomizedSearchCV

In [ ]:
svm_model = SVR(kernel='linear')

In [ ]:
grid_search = RandomizedSearchCV(svm_model,param_grid,scoring='r2',n_jobs=-1)

grid_search.fit(trainX,trainY)

print(grid_search.best_estimator_)

In [ ]:
svm_model = SVR(C=10000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.3, gamma=0.517947467923121,
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

svm_model.fit(trainX,trainY)

pred = svm_model.predict(testX)

In [ ]:
testY_array = np.array(testY).reshape(-1, 1)
pred_array = np.array(pred).reshape(-1, 1)

testY_inverse = scaler.inverse_transform(testY_array)
pred_inverse = scaler.inverse_transform(pred_array)

testY_inverse_flat = testY_inverse.flatten()
pred_inverse_flat = pred_inverse.flatten()

testScore = np.sqrt(mean_squared_error(testY_inverse_flat, pred_inverse_flat))
print('Test Score: %.2f RMSE' % (testScore))



In [ ]:
print('RSquared :','{:.2%}'.format(r2_score(testY,pred)))

In [ ]:
plt.figure(figsize=[8,5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.plot(scaler.inverse_transform(testY.values.reshape(-1, 1)), label='True')
plt.plot(scaler.inverse_transform(pred.reshape(-1, 1)), label='Prediction')
plt.legend()

## Ridge Regression

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
ridge = RidgeCV()

In [ ]:
ridge.fit(trainX,trainY)

Rpred = ridge.predict(testX)

In [ ]:
testY_array = np.array(testY).reshape(-1, 1)
Rpred_array = np.array(Rpred).reshape(-1, 1)

testY_inverse = scaler.inverse_transform(testY_array)
Rpred_inverse = scaler.inverse_transform(Rpred_array)

testY_inverse_flat = testY_inverse.flatten()
Rpred_inverse_flat = Rpred_inverse.flatten()

testScore = np.sqrt(mean_squared_error(testY_inverse_flat, Rpred_inverse_flat))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
print('RSquared :','{:.2%}'.format(r2_score(testY,Rpred)))

In [ ]:
plt.figure(figsize=[8,5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.plot(scaler.inverse_transform(testY.values.reshape(-1, 1)), label='True')
plt.plot(scaler.inverse_transform(Rpred.reshape(-1, 1)), label='Prediction')
plt.legend()

## Hyperparameter Tuning

In [ ]:
normal_price = np.array([float(candles[i][4]) for i in range(1000)])

In [ ]:
data = pd.DataFrame(normal_price.reshape(200,5),columns=['First','Second','Third','Fourth','Target'])

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
x_train_r = df.iloc[:149,:4]
y_train_r = df.iloc[:149,-1]

x_test_r = df.iloc[150:199,:4]
y_test_r = df.iloc[150:199,-1]

In [ ]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2)
tpot.fit(x_train_r, y_train_r)

In [ ]:
tpred = tpot.predict(x_test_r)

In [ ]:
testScore = np.sqrt(mean_squared_error(y_test_r,tpred))
print('Test Score: %.2f RMSE' % (testScore))

print('RSquared :','{:.2%}'.format(r2_score(y_test_r,tpred)))

In [ ]:
plt.figure(figsize=[8,5])
plt.title('Model Fit')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.plot(np.array(y_test_r).reshape(49,), label='True')
plt.plot(tpred, label='Prediction')
plt.legend()

## Trading bot

In [ ]:
check = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_5MINUTE, limit=1000)

check[499]

In [ ]:
index = [996, 997, 998, 999]

candles = np.array([float(check[i][4]) for i in index]).reshape(-1, 1)

candles_scaled = scaler.transform(candles)

model_feed = candles_scaled.reshape(1, 4, 1)

prediction_scaled = model.predict(model_feed)

prediction = scaler.inverse_transform(prediction_scaled.reshape(-1, 1))

predicted_price = prediction[0, 0]
print(predicted_price)

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from binance.client import Client
import os
from dotenv import load_dotenv
import time  
import tensorflow as tf

tf.keras.utils.disable_interactive_logging()

load_dotenv()

api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")

client = Client(api_key, api_secret)

symbol = 'BTCUSDT'
quantity = '0.0001' 
position = None 
index = [996, 997, 998, 999]

trades_df = pd.DataFrame(columns=['buy_price', 'buy_time', 'sell_price', 'sell_time', 'profit', 'profit_before_fees', 'direction', 'sell_reason'])

start_time = datetime.now()

total_profit = 0
total_profit_before_fees = 0
successful_trades = 0
failed_trades = 0

fee_rate = 0.00075  
profit_threshold = 150  
stop_loss_threshold = 150 

time_limit = timedelta(hours=1)

while datetime.now() < start_time + time_limit or position:
    try:
        price = client.get_recent_trades(symbol=symbol)
        candle = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_5MINUTE, limit=1000)
        
        candles = np.array([float(candle[i][4]) for i in index]).reshape(-1, 1)
        candles_scaled = scaler.transform(candles)
        model_feed = candles_scaled.reshape(1, 4, 1)

        predicted_price = float(scaler.inverse_transform(model.predict(model_feed))[0][0])
        current_price = float(price[-1]['price'])

        if not position and datetime.now() < start_time + time_limit:
            if predicted_price >= current_price + profit_threshold:  
                print(f"Predicted price: {predicted_price}, Current price: {current_price}")
                position = 'long'
                buy_price = float(client.get_order_book(symbol=symbol)['asks'][0][0])
                buy_time = datetime.now()
                stop_loss_price = buy_price - stop_loss_threshold
                # client.order_limit_buy(symbol=symbol, quantity=quantity, price=buy_price)
                print('Long position taken @', buy_price, 'USDT, Stop-Loss:', stop_loss_price)

            elif predicted_price <= current_price - profit_threshold:  
                print(f"Predicted price: {predicted_price}, Current price: {current_price}")
                position = 'short'
                sell_price = float(client.get_order_book(symbol=symbol)['bids'][0][0])
                sell_time = datetime.now()
                stop_loss_price = sell_price + stop_loss_threshold
                # client.order_limit_sell(symbol=symbol, quantity=quantity, price=sell_price)
                print('Short position taken @', sell_price, 'USDT, Stop-Loss:', stop_loss_price)

        if position:
            current_time = datetime.now()
            current_price = float(price[-1]['price'])
            sell_reason = ""  

            if position == 'long':
                if current_price < stop_loss_price:
                    sell_reason = "Long position stop-loss triggered"
                    position = None
                elif (current_price - buy_price) >= profit_threshold:
                    sell_reason = "Long position profit target reached"
                    position = None
                elif (current_time - buy_time) >= timedelta(minutes=5):
                    sell_reason = "Long position time limit reached"
                    position = None
                
                if position is None:  
                    sell_price = float(client.get_order_book(symbol=symbol)['bids'][0][0])
                    sell_time = current_time
                    #client.order_limit_sell(symbol=symbol, quantity=quantity, price=sell_price)

            elif position == 'short':
                if current_price > stop_loss_price:
                    sell_reason = "Short position stop-loss triggered"
                    position = None
                elif (sell_price - current_price) >= profit_threshold:
                    sell_reason = "Short position profit target reached"
                    position = None
                elif (current_time - sell_time) >= timedelta(minutes=5):
                    sell_reason = "Short position time limit reached"
                    position = None

                if position is None:  
                    buy_price = float(client.get_order_book(symbol=symbol)['asks'][0][0])
                    buy_time = current_time
                    #client.order_limit_buy(symbol=symbol, quantity=quantity, price=buy_price)

            if position is None:  
                profit_before_fees = (sell_price - buy_price) * float(quantity) if sell_reason.startswith("Long") else (buy_price - sell_price) * float(quantity)
                buy_cost = buy_price * float(quantity) * (1 + fee_rate)
                sell_revenue = sell_price * float(quantity) * (1 - fee_rate)
                profit = sell_revenue - buy_cost

                total_profit += profit
                total_profit_before_fees += profit_before_fees

                if profit > 0:
                    successful_trades += 1
                else:
                    failed_trades += 1

                new_trade = pd.DataFrame({
                    'buy_price': [buy_price],
                    'buy_time': [buy_time],
                    'sell_price': [sell_price],
                    'sell_time': [sell_time],
                    'profit': [profit],
                    'profit_before_fees': [profit_before_fees],
                    'sell_reason': [sell_reason],
                    'direction': ['long' if sell_reason.startswith("Long") else 'short']
                })
                trades_df = pd.concat([trades_df, new_trade], ignore_index=True)
                
                print(f'Sell @Limit Price: {sell_price} USDT, Timestamp: {str(sell_time)}, Reason: {sell_reason}, Profit: {profit} USDT')

    except Exception as e:
        print("An error occurred:", e)
        time.sleep(5) 

print("Total profit after fees:", total_profit, "USDT")
print(trades_df)
